# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.3 MB/s eta 0:00:00


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [37]:
# Data loading functions. Uncomment the one you want to use
import pandas as pd
from worcliver.load_data import load_data


data = load_data()
data_df = pd.DataFrame(data)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 186
The number of columns: 494


### Extracting the label per patient

This label (benign/malignant) is the ground truth per patient.

In [42]:
label = data_df[['label']]

### Pre-processing

### Dividing our data into test-, training- and validation-set

The dataset should be randomly divided into :
* training datasets = 70 % of samples
* test datasets = 20 % of samples
* validation datasets = 10 % of samples

### Finding the missing data

Checking whether the missing data is stored as NaN or 0, finding the columns in which missing data is found as well as the empty features.


*--> dit moet alleen over de trainingsset gedaan worden toch? dat weghalen van de missing data enzo? voor nu is dit dus gedaan over de hele dataset*

In [45]:
#checking whether missing data is stored as NaN
nan_check = data_df.isna()

#conclusion: missing data is not stored as NaN

#checking whether missing data is stored as 0
zero_counts = (data_df == 0).sum()

#conclusion: missing data is stored as 0

zero_counts_columns = zero_counts[zero_counts > 0]
print(f'The number of features in which there is missing data: {zero_counts_columns.count()}')
print(f'The number of missing data is: {sum(zero_counts_columns)}')

empty_features = zero_counts_columns[zero_counts_columns == len(data_df.index)]
print(f'The total number of features containing zeros (so no data) is: {empty_features.count()}')
print(f'The number of features in which there is missing data (excluding empty features): {zero_counts_columns.count()-empty_features.count()}')
print(f'The number of missing data is (excluding empty features): {sum(zero_counts_columns)-empty_features.count()*len(data.index)}')

The number of features in which there is missing data: 22
The number of missing data is: 3039
The total number of features containing zeros (so no data) is: 0
The number of features in which there is missing data (excluding empty features): 22
The number of missing data is (excluding empty features): 3039


### Handling the missing data

The missing data was handled as follows:
1. Deleting the empty features from the data
2. ...

In [46]:
#step 1: deleting the empty features
data_cleaned = data_df.drop(columns=empty_features.index, inplace=True)
print(f'The number of columns after deleting the empty features: {len(data_df.columns)}')

#step 2: 

The number of columns after deleting the empty features: 487


### Feature selection